# Chapter 06: Example Application

**Declarative coding:** Focuses on describing what you want to achieve without specifying how to do it. The system handles the implementation details (e.g., SQL). \

**Imperative coding:** Focuses on describing how to achieve a result through explicit instructions or steps, manipulating the program's state (e.g., C, Java).

**Differences:**
- _Declarative:_ Focuses on the what
- _Imperative:_ Focuses on the how

First Example: 

In [1]:
class Car:
    def __init__(self, name):
        self.make = name

cars = [Car('Jaguar'), Car('Volvo')]

In [3]:
# imperative
makes = []
for i in range(len(cars)):
    makes.append(cars[i].make)
makes

['Jaguar', 'Volvo']

In [6]:
# declarative
makes = list(map(lambda car: car.make, cars))
makes

['Jaguar', 'Volvo']

Second Example:

In [15]:
users = {'test': 'pass123'}
toUser = lambda form: form['username'] if form['username'] in users and users[form['username']]==form['password'] else None 
logIn = lambda x: print(f'User {x} has logged in!') if x in users else print('User or Password Incorrect')

In [17]:
# imperative
def authenticate(form):
  user = toUser(form)
  return logIn(user)

authenticate({'username': 'test', 'password': 'pass123'})

User test has logged in!


In [24]:
from utils import compose

# declarative
authenticate = compose(logIn, toUser)

authenticate({'username': 'test', 'password': 'pass123'})

User test has logged in!
